# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 54 new papers today
          20 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/19 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.11238


extracting tarball to tmp_2209.11238...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.11244


extracting tarball to tmp_2209.11244...

...

Retrieving document from  https://arxiv.org/e-print/2209.11701


extracting tarball to tmp_2209.11701...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.11735


extracting tarball to tmp_2209.11735...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11248-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11248) | **Comparing simulated Milky Way satellite galaxies with observations using  unsupervised clustering**  |
|| <mark>Li-Hsin Chen</mark>, Tilman Hartwig, Ralf S. Klessen, Simon C. O. Glover |
|*Appeared on*| *2022-09-26*|
|*Comments*| *11 pages, 7 figures, 3 tables. Submitted to MNRAS, comments are welcome*|
|**Abstract**| We develop a new analysis method that allows us to compare multi-dimensional observables to a theoretical model. The method is based on unsupervised clustering algorithms which assign the observational and simulated data to clusters in high dimensionality. From the clustering result, a goodness of fit (the p-value) is determined with the Fisher-Freeman-Halton test. We first show that this approach is robust for 2D Gaussian distributions. We then apply the method to the observed MW satellites and simulated satellites from the fiducial model of our semi-analytic code A-SLOTH. We use the following 5 observables of the galaxies in the analysis: stellar mass, virial mass, heliocentric distance, mean stellar metallicity [Fe/H], and stellar metallicity dispersion {\sigma}[Fe/H]. A low p-value returned from the analysis tells us that our A-SLOTH fiducial model does not reproduce the mean stellar metallicity of the observed MW satellites well. We implement an ad-hoc improvement to the physical model and show that the number of dark matter merger trees which have p-values > 0.01 increases from 3 to 6. This method can be extended to data with higher dimensionality easily. We plan to further improve the physical model in A-SLOTH using this method to study elemental abundances of stars in the observed MW satellites. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11602-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11602) | **GRAVITY+ Wide: Towards hundreds of z $\sim$ 2 AGN**  |
|| A. Drescher, et al. -- incl., <mark>L. Kreidberg</mark> |
|*Appeared on*| *2022-09-26*|
|*Comments*| *Proceeding of SPIE Astronomical Telescopes + Instrumentation 2022*|
|**Abstract**| As part of the GRAVITY$^{+}$ project, the near-infrared beam combiner GRAVITY and the VLTI are currently undergoing a series of significant upgrades to further improve the performance and sky coverage. The instrumental changes will be transformational, and for instance uniquely position GRAVITY to observe the broad line region of hundreds of Active Galactic Nuclei (AGN) at a redshift of two and higher. The increased sky coverage is achieved by enlarging the maximum angular separation between the celestial science object (SC) and the off-axis fringe tracking (FT) star from currently 2 arcseconds (arcsec) up to unprecedented 30 arcsec, limited by the atmospheric conditions. This was successfully demonstrated at the VLTI for the first time. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11238-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11238) | **Cosmicflows-4**  |
|| R. Brent Tully, et al. -- incl., <mark>Thomas de Jaeger</mark>, <mark>Joseph B. Jensen</mark> |
|*Appeared on*| *2022-09-26*|
|*Comments*| *38 pages, 24 figures. catalogs available at edd.ifa.hawaii.edu. Accepted to ApJ*|
|**Abstract**| With Cosmicflows-4, distances are compiled for 55,877 galaxies gathered into 38,065 groups. Eight methodologies are employed, with the largest numbers coming from the correlations between the photometric and kinematic properties of spiral galaxies (TF) and elliptical galaxies (FP). Supernovae that arise from degenerate progenitors (SNIa) are an important overlapping component. Smaller contributions come from distance estimates from the surface brightness fluctuations of elliptical galaxies (SBF) and the luminosities and expansion rates of core collapse supernovae (SNII). Cepheid Period-Luminosity Relation (CPLR) and Tip of the Red Giant Branch (TRGB) observations founded on local stellar parallax measurements along with the geometric maser distance to NGC 4258 provide the absolute scaling of distances. The assembly of galaxies into groups is an important feature of the study in facilitating overlaps between methodologies. Merging between multiple contributions within a methodology and between methodologies is carried out with Bayesian Markov chain Monte Carlo procedures. The final assembly of distances is compatible with a value of the Hubble constant of $H_0=75.0$ km s$^{-1}$ Mpc$^{-1}$ with the small statistical error $\pm$ $0.8$ km s$^{-1}$ Mpc$^{-1}$ but a large potential systematic error ~3 km s$^{-1}$ Mpc$^{-1}$. Peculiar velocities can be inferred from the measured distances. The interpretation of the field of peculiar velocities is complex because of large errors on individual components and invites analyses beyond the scope of this study. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11701-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11701) | **Foreground removal and 21 cm signal estimates: comparing different blind  methods for the BINGO Telescope**  |
|| Alessandro Marins, et al. -- incl., <mark>Vincenzo Liccardo</mark>, <mark>Jiajun Zhang</mark> |
|*Appeared on*| *2022-09-26*|
|*Comments*| **|
|**Abstract**| BINGO will observe hydrogen distribution by means of the 21 cm line signal by drift-scan mapping through a tomographic analysis called \emph{Intensity Mapping} (IM) between 980 and 1260 MHz which aims at analyzing Dark Energy using \emph{Baryon Acoustic Oscillations}. In the same frequency range, there are several other unwanted signals as well as instrumental noise, contaminating the target signal. There are many component separation methods to reconstruct signals. Here, we used just three blind methods (FastICA, GNILC and GMCA), which explore different ways to estimate foregrounds' contribution from observed signals from the sky. Subsequently, we estimate 21 cm signal from its mixing with noise. We also analyzed how different number of simulations affect the quality of the estimation, as well as the effect of the binning on angular power spectrum to estimate 21 cm from the mixing with noise. For the BINGO sky range and sensitivity and the foreground model considered in the current simulation, we find that the effective dimension of the foreground subspace leading to best results is equal to three, composed of non-physical templates. At this moment of the pipeline configuration, using 50 or 400 simulations is statistically equivalent. It is also possible to reduce the number of multipoles by half to speed up the process and maintain the quality of results. All three algorithms used to perform foreground removal yielded statistically equivalent results for estimating the 21cm signal when we assume 400 realizations and GMCA and FastICA's mixing matrix dimensions equal to three. However, concerning computational cost in this stage of the BINGO pipeline, FastICA is faster than other algorithms. A new comparison will be necessary when the time-ordered-data and map-making are available. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11735-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11735) | **The GAPS Programme at TNG. XLI. The climate of KELT-9b revealed with a  new approach to high spectral resolution phase curves**  |
|| L. Pino, et al. -- incl., <mark>A. Maggio</mark> |
|*Appeared on*| *2022-09-26*|
|*Comments*| *Recommended for publication on A&A after referee report, awaiting acceptance. 25 pages, 19 figures, 3 tables*|
|**Abstract**| [Abridged] We present a novel method to study the thermal emission of exoplanets as a function of orbital phase at very high spectral resolution, and apply it to investigate the climate of the ultra-hot Jupiter KELT-9b. We combine 3 nights of HARPS-N and 2 nights of CARMENES optical spectra, covering orbital phases between quadratures (0.25 < phi < 0.75), when the planet shows its day-side hemisphere with different geometries. We co-add the signal of thousands of FeI lines through cross-correlation, which we map to a likelihood function. We investigate the phase-dependence of: (i) the line depths of FeI, and (ii) their Doppler shifts, by introducing a new method that exploits the very high spectral resolution of our observations. We confirm a previous detection of FeI emission and demonstrate a combined precision of 0.5 km s-1 on the orbital properties of KELT-9b. By studying the phase-resolved Doppler shift of FeI lines, we detect an anomaly in the planet's orbital radial velocity well-fitted with a slightly eccentric orbit (e = 0.016$\pm$0.003, w = 150$^{+13\circ}_{-11},~5\sigma$ preference). However, we argue that such anomaly can be explained by a day-night wind of a few km s-1 blowing neutral iron gas. Additionally, we find that the FeI emission line depths are symmetric around the substellar point within 10 deg ($2\sigma$). We show that these results are qualitatively compatible with predictions from general circulation models for ultra-hot Jupiter planets. Very high-resolution spectroscopy phase curves have the sensitivity to reveal a phase dependence in both the line depths and their Doppler shifts throughout the orbit. They are highly complementary to space-based phase curves obtained with HST and JWST, and open a new window into the still poorly understood climate and atmospheric structure of the hottest planets known. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11496-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11496) | **Comparing the Super-Eddington accretion of SMC X-3 and RX J0209.6-7427  with Swift J0243.6+6124**  |
|| <mark>Jiren Liu</mark>, et al. -- incl., <mark>Shuang-Nan Zhang</mark> |
|*Appeared on*| *2022-09-26*|
|*Comments*| *9 pages, 5 figs, to appear on MNRAS*|
|**Abstract**| We study the giant outbursts of SMC X-3 and RX J0209.6-7427 to compare their super-Eddington accretion regime with that of Swift J0243.6+6124. The high double-peak profile of SMC X-3 is found to be 0.25 phase offset from that below $2.3\times10^{38}$erg\,s$^{-1}$, which is similar to Swift J0243 (happened around $0.9\times10^{38}$erg\,s$^{-1}$). The profile of RX J0209 shows a similar 0.25 phase offset between high double-peak and low double-peak around $1.25\times10^{38}$erg\,s$^{-1}$. The 0.25 phase offset corresponds to a 90 degree angle change of the emission beam and strongly supports for a transition from a fan beam to a pencil beam. Their critical luminosities imply a surface magnetic field $\sim4\times10^{13}$ G and $2\times10^{13}$ G for SMC X-3 and RX J0209, respectively, based on the recently measured cyclotron line of Swift J0243. The spin-up rate and luminosity of SMC X-3 follows a relation of $\dot{\nu}\propto L^{0.94\pm0.03}$, while that of RX J0209 follows $\dot{\nu}\propto L^{1.00\pm0.03}$, which are similar to Swift J0243 and consistent with the prediction of a radiation-pressure-dominated (RPD) disk. These results indicate that accretion columns are indeed formed above Eddington luminosity, and the population of ULXPs likely corresponds to X-ray pulsars of highest magnetic field. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'utf-8' codec can't decode byte 0xc3 in position 45384: invalid continuation byte</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11264-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11264) | **Titan Atmospheric Chemistry Revealed by Low-temperature N2-CH4 Plasma  Discharge Experiments**  |
|| <mark>Chao He</mark>, et al. -- incl., <mark>Joseph Serigano</mark> |
|*Appeared on*| *2022-09-26*|
|*Comments*| *Accepted in ACS Earth and Space Chemistry, 6 figures*|
|**Abstract**| Chemistry in Titan's N2-CH4 atmosphere produces complex organic aerosols. The chemical processes and the resulting organic compounds are still far from understood, although extensive observations, laboratory, and theoretical simulations have greatly improved physical and chemical constraints on Titan's atmosphere. Here, we conduct a series of Titan atmosphere simulation experiments with N2-CH4 gas mixtures and investigate the effect of initial CH4 ratio, pressure, and flow rate on the production rates and composition of the gas and solid products at a Titan relevant temperature (100 K) for the first time. We find that the production rate of the gas and solid products increases with increasing CH4 ratio. The nitrogen-containing species have much higher yield than hydrocarbons in the gas products, and the N-to-C ratio of the solid products appears to be the highest compared to previous plasma simulations with the same CH4 ratio. The greater degree of nitrogen incorporation in the low temperature simulation experiments suggests temperature may play an important role in nitrogen incorporation in Titan's cold atmosphere. We also find that H2 is the dominant gas product and serves as an indicator of the production rate of new organic molecules in the experiment, and that CH2NH may greatly contribute to the incorporation of both carbon and nitrogen into the solid particles. The pressure and flow rate affect the amount of time of the gas mixture exposed to the energy source and therefore impact the N2-CH4 chemistry initiated by the plasma discharge, emphasizing the influence of the energy flux in Titan atmospheric chemistry. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11685-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11685) | **Opportunities for Technosignature Science in the Planetary Science and  Astrobiology Decadal Survey**  |
|| Jacob Haqq-Misra, et al. -- incl., <mark>Joseph Lazio</mark> |
|*Appeared on*| *2022-09-26*|
|*Comments*| *A report from the First Penn State SETI Symposium*|
|**Abstract**| Solar system exploration provides numerous possibilities for advancing technosignature science. The search for life in the solar system includes missions designed to search for evidence of biosignatures on other planetary bodies, but many missions could also attempt to search for and constrain the presence of technology within the solar system. Technosignatures and biosignatures represent complementary approaches toward searching for evidence of life in our solar neighborhood, and beyond. This report summarizes the potential technosignature opportunities within ongoing solar system exploration and the recommendations of the "Origins, Worlds, and Life" Planetary Science and Astrobiology Decadal Survey. We discuss opportunities for constraining the prevalence of technosignatures within the solar system using current or future missions at negligible additional cost, and we present a preliminary assessment of gaps that may exist in the search for technosignatures within the solar system. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.11248.md
    + _build/html/tmp_2209.11248/./figs/hist_comp.png
    + _build/html/tmp_2209.11248/./figs/fid_nc5_mstellarmeanFeH_Agglomerative_-2.2.png
    + _build/html/tmp_2209.11248/./figs/fid_Agglomerative_inditree_p_nc5.png
exported in  _build/html/2209.11602.md
    + _build/html/tmp_2209.11602/./figures/gwide_upgrades.png
    + _build/html/tmp_2209.11602/./figures/ddl_m3_replacement.png
    + _build/html/tmp_2209.11602/./figures/vis_loss_all_Gwide_20220819_v1.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\angstrom}{\textup{\AA}}$
$\newcommand{\asloth}{\textsc{a-sloth}\xspace}$
$\newcommand{\ctp}{\textit{Caterpillar}\xspace}$
$\newcommand{\msun}{{\rm M}_\odot}$
$\newcommand{\feh}{\mathrm{[Fe/H]}}$
$\newcommand{\tilman}[1]{\textcolor{green}{{#1}}}$
$\newcommand{\lhnote}[1]{\textcolor{red}{{#1}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\angstrom$}{\textup{\AA}}$
$\newcommand{$\asloth$}{$\textsc${a-sloth}$\xspace$}$
$\newcommand{$\ctp$}{\textit{Caterpillar}$\xspace$}$
$\newcommand{$\msun$}{{\rm M}_\odot}$
$\newcommand{$\feh$}{\mathrm{[Fe/H]}}$
$\newcommand{$\tilman$}[1]{\textcolor{green}{{#1}}}$
$\newcommand{$\lhnote$}[1]{\textcolor{red}{{#1}}}$</div>



<div id="title">

# Comparing simulated Milky Way satellite galaxies with observations using unsupervised clustering

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.11248-b31b1b.svg)](https://arxiv.org/abs/2209.11248)<mark>Appeared on: 2022-09-26</mark> - _11 pages, 7 figures, 3 tables. Submitted to MNRAS, comments are welcome_

</div>
<div id="authors">

<mark><mark>Li-Hsin Chen</mark></mark>, Tilman Hartwig, Ralf S. Klessen, Simon C. O. Glover

</div>
<div id="abstract">

**Abstract:** We develop a new analysis method that allows us to compare multi-dimensional observables to a theoretical model. The method is based on unsupervised clustering algorithms which assign the observational and simulated data to clusters in high dimensionality. From the clustering result, a goodness of fit (the p-value) is determined with the Fisher-Freeman-Halton test.We first show that this approach is robust for 2D Gaussian distributions. We then apply the method to the observed MW satellites and simulated satellites from the fiducial model of our semi-analytic code$\asloth$. We use the following 5 observables of the galaxies in the analysis: stellar mass, virial mass, heliocentric distance, mean stellar metallicity$\feh$, and stellar metallicity dispersion$\sigma_$\feh$$.A low p-value returned from the analysis tells us that our$\asloth$fiducial model does not reproduce the mean stellar metallicity of the observed MW satellites well. We implement an ad-hoc improvement to the physical model and show that the number of dark matter merger trees which have p-values > 0.01 increases from 3 to 6.This method can be extended to data with higher dimensionality easily. We plan to further improve the physical model in$\asloth$using this method to study elemental abundances of stars in the observed MW satellites.

</div>

<div id="div_fig1">

<img src="tmp_2209.11248/./figs/hist_comp.png" alt="Fig4" width="100%"/>

**Figure 4. -** Normalised, cumulative histogram of the five physical quantities used in the analysisNormalised, cumulative histogram of the five physical quantities used in the analysis from our fiducial model, improved model, and the observation. From top to bottom: stellar mass, helicocentric distance, mean stellar $\feh$, standard deviation of stellar $\feh$, and the virial mass of the halo. The observation is plotted in green, the $\asloth$ fiducial model is plotted in blue, and the improved model is plotted in brown.  (*fig:hist_fid*)

</div>
<div id="div_fig2">

<img src="tmp_2209.11248/./figs/fid_nc5_mstellarmeanFeH_Agglomerative_-2.2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Example of clustering results from $\asloth$ fiducial model An example of unsupervised clustering from the fiducial model of $\asloth$ and the observed satellites, which is projected onto the $\langle$\feh$\rangle$-${M_*}$ space. The data is normalised in each dimension before we apply the unsupervised clustering and the means are shown in bold font. In this analysis we use Agglomerative with 5 clusters. Galaxies that are classified in different clusters are plotted with different colours. Observed MW satellites are shown in squares and $\asloth$ simulated MW satellites are shown in circles. The p-value for this example is $10^{-2.2}$. (*fig:fid_clus*)

</div>
<div id="div_fig3">

<img src="tmp_2209.11248/./figs/fid_Agglomerative_inditree_p_nc5.png" alt="Fig2" width="100%"/>

**Figure 2. -** The p-values from 30 $\ctp$ treesThe p-values from 30 Caterpillar trees as individual datasets and the p-value from the Ensemble (data from 30 trees combined before conducting the analysis). Mean and 1 standard deviation from 100 runs of the analysis are shown with error bars. (*fig:indi_pv*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\gp}{GRAVITY^+}$
$\newcommand{\gw}{GRAVITY Wide}$
$\newcommand{\baselinestretch}{1.0}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\gp$}{GRAVITY^+}$
$\newcommand{$\gw$}{GRAVITY Wide}$
$\newcommand{$\baselinestretch$}{1.0}$</div>



<div id="title">

#  2 AGN

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.11602-b31b1b.svg)](https://arxiv.org/abs/2209.11602)<mark>Appeared on: 2022-09-26</mark> - _Proceeding of SPIE Astronomical Telescopes + Instrumentation 2022_

</div>
<div id="authors">

A. Drescher, et al. -- incl., <mark><mark>L. Kreidberg</mark></mark>

</div>
<div id="abstract">

**Abstract:** As part of the$\gp$project, the near-infrared beam combiner GRAVITY and the VLTI are currently undergoing a series of significant upgrades to further improve the performance and sky coverage. The instrumental changes will be transformational, and for instance uniquely position GRAVITY to observe the broad line region of hundreds of Active Galactic Nuclei (AGN) at a redshift of two and higher. The increased sky coverage is achieved by enlarging the maximum angular separation between the celestial science object (SC) and the off-axis fringe tracking (FT) star from currently 2 arcseconds (arcsec) up to unprecedented 30 arcsec, limited by the atmospheric conditions. This was successfully demonstrated at the VLTI for the first time.

</div>

<div id="div_fig1">

<img src="tmp_2209.11602/./figures/gwide_upgrades.png" alt="Fig1" width="100%"/>

**Figure 1. -** Schematic overview of the VLTI switchyard and beam routing towards GRAVITY. In total, eight beams from four telescopes arrive at the switchyward: four beams dedicated to the FT target (beam A, black) and four beams dedicated to the SC target (beam B, blue). The figure shows the configuration for $\gw$  observations with the periscopes in "IN" position. The switchyward reconfiguration relays the B beams coming from the beam compressors via the upgraded PRIMA DDLs to the periscopes to merge them into the A beams. (*fig:schematic-switchyard*)

</div>
<div id="div_fig2">

<img src="tmp_2209.11602/./figures/ddl_m3_replacement.png" alt="Fig3" width="100%"/>

**Figure 3. -** Replacement of the M3 mirrors of the PRIMA DDLs. The top figure shows a drawing of the PRIMA DDLs and the location of the M3 mirrors. The bottom left picture shows the piezo driven tip/tilt assembly that is accessible from the backside of the PRIMA DDLs. The red circles indicate the shim locations. The bottom right picture shows the newly fabricated M3 in the thermally clamped mount (here for a prototype, the final mounts were anodized black). (*fig:ddl_m3_replacement*)

</div>
<div id="div_fig3">

<img src="tmp_2209.11602/./figures/vis_loss_all_Gwide_20220819_v1.png" alt="Fig5" width="100%"/>

**Figure 5. -** Observed contrast loss versus FT--SC separation. The three curves indicate the contrast loss for different seeing and isoplanatic angle values as expected from atmospheric turbulence following Ref. \citenum{elhalkouj2008} for the 1.8m ATs. Both the seeing, $\epsilon$, and the isoplanatic angle, $\theta_0$, are defined at 500 nm. (*fig:visloss_all_gwide*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

44  publications in the last 7 days.
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers